In [8]:
import csv
import string
import numpy as np

In [9]:
words = {}
wordIndex = 0
punctuations = string.punctuation + '\n'

In [28]:
def getProperSentence(sentence):
    returnSentence = sentence.lower()
    replace_punctuation = string.maketrans(punctuations, ' '*len(punctuations))
    returnSentence = returnSentence.translate(replace_punctuation)
    return returnSentence

In [29]:
with open('train.csv','rb') as train:
    reader = csv.reader(train)
    for row in reader:
        message = getProperSentence(row[4])
        for word in message.split(' '):
            if not word.isdigit():
                if word not in words:
                    words[word] = wordIndex
                    wordIndex = wordIndex + 1

In [30]:
wordIndex

2229

In [65]:
def makeFeatureVector(sentence):
    tempSen = getProperSentence(sentence)
    retVal = np.zeros(wordIndex)
    for word in tempSen.split(' '):
        if word.isdigit():
            continue
        indexOfWord = words[word]
        retVal[indexOfWord] = 1
    return retVal

In [128]:
def returnFeatureSet():
    index = 0
    ret = np.array([])
    with open('train.csv','rb') as train:
        reader = csv.reader(train)
        for row in reader:        
            featureVector = makeFeatureVector(row[4])
#             print featureVector
            if index == 0:
                ret = np.hstack((ret,featureVector))
                index = index + 1
            else:
                ret = np.vstack((ret, featureVector))
    return ret

In [130]:
featureVectorSet = returnFeatureSet()

In [132]:
from sklearn.cluster import KMeans

In [208]:
kmeans = KMeans(n_clusters=100, random_state=0).fit(featureVectorSet)

In [209]:
kmeans.labels_

array([99,  1,  1,  1,  1,  1, 60,  1,  4, 14,  1,  1, 90, 66, 26, 66, 14,
       86, 64, 14, 14,  1, 45, 26, 81, 39,  1,  1, 10, 90,  1, 81, 42, 28,
       28,  1,  1, 14,  1,  1, 66,  1, 84,  1, 43,  1,  1, 38, 60, 72, 23,
        1,  1,  1, 45, 10,  3,  3, 11, 18,  1, 10,  3, 40, 10,  3, 39,  1,
       28,  1,  1, 89,  4,  1, 23, 23, 64, 86, 39, 28, 78, 36, 36,  1, 78,
        1, 82,  5,  1, 12,  1, 40, 53, 31, 76,  8, 84,  1, 14, 28, 28, 28,
       28, 13,  1, 66,  1, 15, 84, 67,  4, 25, 71, 52, 52, 20,  1, 18, 19,
        1, 19,  1,  1,  7, 37,  7,  7, 66, 58,  8,  6,  7,  2, 37,  7, 58,
       58,  1,  1,  1,  8, 84, 59,  6,  7,  2, 18,  2,  7, 37,  7,  2, 11,
        1, 44, 12, 45, 39, 20,  9, 58,  6,  7,  2, 58,  1, 92,  7, 83,  6,
        7,  2,  6,  6,  7,  2,  1, 73,  1,  4,  1,  8,  7,  7,  2, 12, 58,
       73, 10, 43,  1,  8,  7,  2, 73, 31,  1, 55,  2,  1,  8,  7,  2,  1,
       41, 43, 98, 35, 81,  1, 39,  1, 19, 56, 98, 10, 36, 36, 21, 69,  1,
        1, 71, 39, 19,  1

In [210]:
messageList = []

In [211]:
def populateMessageList():
    with open('train.csv','rb') as train:
        reader = csv.reader(train)
        for row in reader:
            message =row[4]
            messageList.append(message)
populateMessageList()

In [212]:
messageList[1:5]

['Buy till Sunday & get EXTRA 300-500 OFF over& above all offers@ www.lenskart.com/cust or Visit Store.1st Time Ever:300 OFF +1st Frame Free; 500 OFF+ 2 for 2500',
 'nitish chandra has requested Rs.5000.0 from you. Click on this link to pay: http://m.p-y.tm/stom',
 'Faasos App Code : 4893 . For verifying your phone number, not some secret treasure.',
 'Faasos App Code : 1752 . For verifying your phone number, not some secret treasure.']

In [184]:
def getClusters(clusterIndices, messageList, requiredIndex):
    retVal = []
    for i in range(len(clusterIndices)):
        if clusterIndices[i] == requiredIndex:
            retVal.append(messageList[i])
    return retVal

In [228]:
getClusters(kmeans.labels_, messageList, 76)

['Your trx is debited to HDFC Bank CREDIT Card for Rs. 199.00 in MUMBAI at HOTSTAR60238 on 2016-11-06:11:24:05.This is not an authenticated trx as per RBI Mandate effective 1 May 12.',
 'Ur transaction on HDFC Bank CREDIT Card ending 7109 for Rs. 1504.00  has been credited/reversed by PAYUPAISA47779 on 2016-10-26:15:58:31',
 'Ur transaction on HDFC Bank CREDIT Card ending 7109 for Rs. 5.00  has been credited/reversed by HOTSTAR on 2016-10-13:16:01:54']

In [206]:
def getClusterSize(X):
    U,S,V = np.linalg.svd(X)
    total = 0.0
    for i in range(len(S)):
        total += S[i]*S[i];
    spectral_sum = 0.0
    optimal_cluster_size = len(S)
    for i in range(len(S)):
        spectral_sum += S[i]*S[i];
        if(spectral_sum / total > 0.80):
            optimal_cluster_size = i+1
            break
    return optimal_cluster_size

In [207]:
getClusterSize(featureVectorSet)

117

In [224]:
kmeans.labels_[18]

64